### Feature extraction

In [1]:
import tensorflow as tf
from keras.utils import load_img
from keras.utils import img_to_array
from keras.applications.resnet import preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
import pandas as pd
from PIL import Image
import PIL
from numpy import asarray
import cv2
from matplotlib import pyplot
import numpy as np

In [24]:
autoencoder = load_model(r'C:\Users\rocci\OneDrive\Desktop\Università\DL\myCode\autoencoder')
autoencoder.layers.pop()
autoencoder = Model(inputs = autoencoder.inputs, outputs = autoencoder.layers[-8].output)


In [ ]:
df = pd.DataFrame()
for row in range(120) :
    image = load_img(r'C:\Users\rocci\OneDrive\Desktop\Università\DL\DiagClassifierTransferLearning-main\DiagClassifierTransferLearning-main\Paintings\row' + str(row) + '.png', target_size = (288, 288))
    image = PIL.ImageOps.grayscale(image)
    image = img_to_array(image)
    image = np.array(image, dtype = np.float32)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    features = autoencoder.predict(image)
    flattened = features.reshape(1, 10368)
    print(flattened.shape)

    df1 = pd.DataFrame(flattened)
    df =  pd.concat([df, df1], ignore_index=True)

users_df = pd.read_csv(r"C:\Users\rocci\OneDrive\Desktop\Università\DL\DiagClassifierTransferLearning-main\DiagClassifierTransferLearning-main\users.txt", sep = "\t", dtype = str)
df["diag"]=users_df["diag"]
df.to_csv(r"C:\Users\rocci\OneDrive\Desktop\Università\DL\myCode\AutoencoderFeatureExtraction.csv")

In [3]:
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

features_df = pd.DataFrame()
features_df = pd.read_csv(r"C:\Users\rocci\OneDrive\Desktop\Università\DL\myCode\features2.csv")
#features_df = features_df.drop(["Unnamed: 0"], axis=1)

train, test = train_test_split(features_df, test_size=20)
trainY = train["diag"]
trainX = train.drop(["diag"], axis=1)
testY = test["diag"]
testX = test.drop(["diag"], axis=1)

trainX
trainY

105    0
111    0
19     1
109    0
106    0
      ..
56     1
91     0
110    0
50     0
60     1
Name: diag, Length: 100, dtype: int64

In [4]:
ridModel = RidgeClassifier(alpha=100)
ridModel.fit(trainX, trainY)

print(ridModel.score(testX,testY))

0.85


In [5]:
logModel = LogisticRegression()
logModel.fit(trainX, trainY)

print(logModel.score(testX,testY))

0.8


c:\Users\rocci\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
tree = DecisionTreeClassifier()
tree.fit(trainX, trainY)
print(tree.score(testX,testY))

0.45


In [7]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(trainX,trainY)
print(clf.score(testX,testY))

0.9


In [8]:
import statistics
accRidge = []
accLogReg = []
accDecTree = []
accRnFr = []
ridModel = RidgeClassifier(alpha=100)
logModel = LogisticRegression()
tree = DecisionTreeClassifier()
clf=RandomForestClassifier(n_estimators=100)
for i in range(1000):
    train, test = train_test_split(features_df, test_size=20)
    trainY = train["diag"]
    trainX = train.drop(["diag"], axis=1)
    testY = test["diag"]
    testX = test.drop(["diag"], axis=1)


    ridModel.fit(trainX, trainY)
    accRidge.append(ridModel.score(testX, testY))
    logModel.fit(trainX, trainY)
    accLogReg.append(logModel.score(testX,testY))
    tree.fit(trainX, trainY)
    accDecTree.append(tree.score(testX,testY))
    clf.fit(trainX,trainY)
    accRnFr.append(clf.score(testX,testY))
print(statistics.mean(accRidge), statistics.mean(accLogReg), statistics.mean(accDecTree), statistics.mean(accRnFr))

c:\Users\rocci\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\rocci\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.7826 0.75165 0.66695 0.84315


In [9]:
print("Media Acc RidgeClassifier: " + str(statistics.mean(accRidge)))
print("Accuracy piu elevata " + str(max(accRidge)))
print("Accuracy piu bassa " + str(min(accRidge)))
print("---------------------------------------------------")
print("Media Acc LogisticRegression: " + str(statistics.mean(accLogReg)))
print("Accuracy piu elevata " + str(max(accLogReg)))
print("Accuracy piu bassa " + str(min(accLogReg)))
print("---------------------------------------------------")
print("Media Acc DecisionTreeClassifier: " + str(statistics.mean(accDecTree)))
print("Accuracy piu elevata " + str(max(accDecTree)))
print("Accuracy piu bassa " + str(min(accDecTree)))
print("---------------------------------------------------")
print("Media Acc RandomForestClassifier: " + str(statistics.mean(accRnFr)))
print("Accuracy piu elevata " + str(max(accRnFr)))
print("Accuracy piu bassa " + str(min(accRnFr)))

Media Acc RidgeClassifier: 0.7826
Accuracy piu elevata 1.0
Accuracy piu bassa 0.45
---------------------------------------------------
Media Acc LogisticRegression: 0.75165
Accuracy piu elevata 1.0
Accuracy piu bassa 0.4
---------------------------------------------------
Media Acc DecisionTreeClassifier: 0.66695
Accuracy piu elevata 0.9
Accuracy piu bassa 0.35
---------------------------------------------------
Media Acc RandomForestClassifier: 0.84315
Accuracy piu elevata 1.0
Accuracy piu bassa 0.55
